# 元类
***

有这么一句话，**python中的一切都是对象**。所以用来创建对象的`Class`，本身也是对象，而它们的抽象就是`type()`，称作元类。

而静态语言和动态语言最大的不同，就是函数和类的定义，不是编译的时候定义的，而是运行时动态创建的。

## type()

In [1]:
class A:
    def __init__(self):
        self.a = 0
    def fuc(self):
        print("this is a")

In [2]:
a = A()
print(type(a))
print(type(A))

<class '__main__.A'>
<class 'type'>


看到，A类型的类(抽象)就是`type`

In [3]:
type(A())

__main__.A

这里就可以说class的定义是运行时动态创建的

`type()`不光可以返回类型，还可以创建类型

In [4]:
def func():
    print("this is func")

A = type('A', (object,), dict(afunc=func))

In [5]:
a = A()
print(type(a))
print(type(A))
a.afunc()

<class '__main__.A'>
<class 'type'>


TypeError: func() takes 0 positional arguments but 1 was given

这里有错，我认为时因为在创建类的时候，所有类的成员方法第一个参数都是`self`，所以在调用这个方法的时候自动传入了`self`也就是a。

所以在定义方法的时候要加入参数`self`

In [8]:
def func(self):
    print("this is func")

A = type('A', (object,), dict(afunc=func))
a = A()
print(type(a))
print(type(A))
a.afunc()

<class '__main__.A'>
<class 'type'>
this is func


定义一个类，`type()`需要传入三个参数type(object_or_name, bases, dict)：
    1. name：一个字符串作为class的名称
    2. bases：继承的父类。是一个元组，可以继承多个父类。
    3. dict：添加的额外参数。一个字典，传入方法或者属性。比如这里就是将已有函数`func`绑定到`afunc`。 

## metaclass
***
先定义metaclass创建元类，然后再用元类创建实例，也就是定义类。

先创建一个metaclass类，因为metaclass是类的模板，而python的元类又是`type`所以要继承type

In [12]:
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        attrs['a'] = "im a"
        return type.__new__(cls, name, bases, attrs)

解释以下`__new__()`方法的几个参数：
    1. cls：当前准备创建的类的对象
    2. name：类的名称
    3. bases：继承的父类的集合
    4. attrs：类的方法属性集合

这里的lambda表示方法`add`是以`self`和`value`作为入口参数，函数体是self.append(value)。

然后就可以用这个类作为元类，创建其他的类。

In [10]:
class MyList(list, metaclass=ListMetaclass):
    pass

当我们传入关键字参数`metaclass`时，就会对调用`__new__()`方法来创建类对象。ListMetaclass的return就是返回一个类。

现在可以调用这个类的`add`方法。

In [16]:
l = MyList()
l.add("asd")
l

['asd']

不知道为什么jupyter输出`l.a`会报错，而pycharm不会或者命令行等别的环境不会。

In [17]:
l.a

AttributeError: 'MyList' object has no attribute 'a'

在我的代理池项目中的[爬虫部分](https://github.com/jiangyuwei666/ProxyPool/blob/master/spider/proxy_spider.py)就用到了元类。主要是利用元类得到所以爬取方法的名称，再利用反射调用。

ORM(object relational mapping)对象-关系映射，就是把关系数据库一样映射为一个对象，也就是一个类一个表，这个时候就可以直接动态定义所有的表的数据的类型，。但是我的mysql账户密码忘记了。并且一时半会没找到方法修改。。。